I use this file to write new ENVI data.

In [36]:
from misc.sen2 import writeENVI
from raster import Raster
import numpy as np

This is where the data will be written.

In [37]:
output_file = 'test_C11659/L1_0609_0904.bin'

These are files where new data are calculated

In [38]:
pre_fire_f = 'L1_C11659/S2C_MSIL1C_20250609T192931_N0511_R142_T09UYU_20250609T230604_cloudfree.bin'

post_fire_f = 'L1_C11659/S2C_MSIL1C_20250904T191931_N0511_R099_T09UYU_20250904T225142_cloudfree.bin'

Perform Calculations

In [39]:
#1. pre and post fire objects

pre_fire_obj = Raster(pre_fire_f)
pre_fire_dat = pre_fire_obj.read_bands('all')

post_fire_obj = Raster(post_fire_f)
post_fire_dat = post_fire_obj.read_bands('all')

In [40]:
# #2. difference product

from change_detection import change_detection

change_dat = change_detection(
    pre_X=pre_fire_dat,
    post_X=post_fire_dat
)

In [41]:
# #3. dNBR SWIR B12

from barc import dNBR

_, _, dnbr12 = dNBR(
    NIR_pre=pre_fire_obj.get_band(8),
    SWIR_pre=pre_fire_obj.get_band(12),

    NIR_post=post_fire_obj.get_band(8),
    SWIR_post=post_fire_obj.get_band(12)
)

In [42]:
#4. dNBR SWIR 11

from barc import dNBR, NBR

_, _, dnbr11 = dNBR(
    NIR_pre=pre_fire_obj.get_band(8),
    SWIR_pre=pre_fire_obj.get_band(11),

    NIR_post=post_fire_obj.get_band(8),
    SWIR_post=post_fire_obj.get_band(11)
)

# nbr11 = NBR(
#     NIR=fire_obj.get_band(8),
#     SWIR=fire_obj.get_band(11)
# )

# nbr12 = NBR(
#     NIR=fire_obj.get_band(8),
#     SWIR=fire_obj.get_band(12)
# )

Merge data and band info

In [43]:
new_dat = np.dstack([
    pre_fire_dat,
    post_fire_dat,
    change_dat,
    dnbr11,
    dnbr12
])

In [44]:
new_dat.shape

(5490, 5490, 14)

In [45]:
new_band_names = [
    'B12 pre',
    'B11 pre',
    'B9 pre',
    'B8 pre',

    'B12 pst',
    'B11 pst',
    'B9 pst',
    'B8 pst',

    'B12 diff',
    'B11 diff',
    'B9 diff',
    'B8 diff',

    'dnbr11',
    'dnbr12'
]

Write new data to file

In [46]:
writeENVI(
    output_filename = output_file,
    data = new_dat,
    mode ='new',
    ref_filename = pre_fire_f,
    band_names = new_band_names
)